In [0]:
service_credential = dbutils.secrets.get(scope="ws-scope",key="weather-sp-secret")

spark.conf.set("fs.azure.account.auth.type.jcystorage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.jcystorage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.jcystorage.dfs.core.windows.net", "a2210f7a-e661-45c1-98f4-af64e6d2df9f")
spark.conf.set("fs.azure.account.oauth2.client.secret.jcystorage.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.jcystorage.dfs.core.windows.net", "https://login.microsoftonline.com/a3b377f7-39e7-4ca3-bf66-ea94bbdcdbf3/oauth2/token")

In [0]:
landing = dbutils.fs.ls("abfss://weather-container-landing@jcystorage.dfs.core.windows.net/")

from pyspark.sql.functions import input_file_name, regexp_extract
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.schemaLocation", "abfss://weather-container-landing@jcystorage.dfs.core.windows.net/weather-container-landing")
      .load("abfss://weather-container-landing@jcystorage.dfs.core.windows.net/weather-container-landing"))

df = df.withColumn("datatimestamp", regexp_extract(input_file_name(), r"/(\d{8})/", 1))
display(df)





# (df.writeStream
#    .format("delta")
#    .option("checkpointLocation", "/mnt/checkpoints/table")
#    .outputMode("append")
#    .table("mi_tabla_destino"))
   
